In [ ]:

import sagemaker
from sagemaker.pytorch import PyTorch
from sagemaker.huggingface import HuggingFace

In [ ]:
session = sagemaker.Session()
bucket = session.default_bucket()
role = sagemaker.get_execution_role()
region = session.boto_region_name()
raw_input_path = f"s3://{bucket}/imdb/processing/raw/small/raw.csv"

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {bucket}")
print(f"sagemaker session region: {region}")

## BYO Docker

In [ ]:
!mkdir docker

This is the Dockerfile to create the processing container. Install `pandas` and `scikit-learn` into it. You can install your own dependencies.

In [ ]:
%%writefile docker/Dockerfile

FROM python:3.7-slim-buster

RUN pip3 install pandas scikit-learn transformers==4.4.2 torch
ENV PYTHONUNBUFFERED=TRUE

ENTRYPOINT ["python3"]

This block of code builds the container using the `docker` command, creates an Amazon Elastic Container Registry (Amazon ECR) repository, and pushes the image to Amazon ECR.

In [ ]:
import boto3

account_id = boto3.client("sts").get_caller_identity().get("Account")
ecr_repository = "sagemaker-pytorch-processing-container"
tag = ":latest"

uri_suffix = "amazonaws.com"
processing_repository_uri = "{}.dkr.ecr.{}.{}/{}".format(
    account_id, region, uri_suffix, ecr_repository + tag
)

In [ ]:
# Create ECR repository and push docker image
!docker build -t $ecr_repository docker
!$(aws ecr get-login --region $region --registry-ids $account_id --no-include-email)
!aws ecr create-repository --repository-name $ecr_repository
!docker tag {ecr_repository + tag} $processing_repository_uri
!docker push $processing_repository_uri

The `ScriptProcessor` class lets you run a command inside this container, which you can use to run your own script.

## Data pre-processing

In [ ]:
from sagemaker.processing import ScriptProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput

script_processor = ScriptProcessor(
    command=["python3"],
    image_uri=processing_repository_uri,
    role=role,
    instance_count=1,
    instance_type="ml.t3.medium",
)

Run the same `preprocessing.py` script you ran above, but now, this code is running inside of the Docker container you built in this notebook, not the scikit-learn image maintained by Amazon SageMaker. You can add the dependencies to the Docker image, and run your own pre-processing, feature-engineering, and model evaluation scripts inside of this container.

In [ ]:
script_processor.run(
    code="source/preprocessing.py",
    inputs=[ProcessingInput(source=raw_input_path, destination="/opt/ml/processing/input")],
    outputs=[
        ProcessingOutput(output_name="train", source="/opt/ml/processing/train"),
        ProcessingOutput(output_name="test", source="/opt/ml/processing/test"),
    ],
    arguments=["--train-test-split-ratio", "0.2", "--model_name", "distilbert-base-uncased"],
)
script_processor_job_description = script_processor.jobs[-1].describe()
output_config = script_processor_job_description["ProcessingOutputConfig"]
for output in output_config["Outputs"]:
    if output["OutputName"] == "train":
        preprocessed_training_data = output["S3Output"]["S3Uri"]
    if output["OutputName"] == "test":
        preprocessed_test_data = output["S3Output"]["S3Uri"]

## Training

In [ ]:
# hyperparameters, which are passed into the training job
hyperparameters = {'epochs': 1,
                   'per_device_train_batch_size': 32,
                   'model_name': 'distilbert-base-uncased'
                   }

# create the Estimator
estimator = HuggingFace(
    entry_point='train.py',
    source_dir='./source',
    instance_type='ml.g4dn.xlarge',  # Note: needs to be an instance with gpu
    instance_count=1,
    role=role,
    transformers_version='4.4',
    pytorch_version='1.6',
    py_version='py36',
    hyperparameters=hyperparameters
)
estimator.fit({'train': preprocessed_training_data, 'test': preprocessed_test_data})

In [ ]:
# model_data = estimator.model_data
model_data = 's3://sagemaker-eu-central-1-611215368770/pytorch-training-2021-06-03-16-49-22-917/model.tar.gz'

## Model Evaluation

`evaluation.py` is the model evaluation script. Since the script also runs using scikit-learn as a dependency,  run this using the `SKLearnProcessor` you created previously. This script takes the trained model and the test dataset as input, and produces a JSON file containing classification evaluation metrics, including precision, recall, and F1 score for each label, and accuracy and ROC AUC for the model.


In [ ]:
import json
from sagemaker.s3 import S3Downloader

script_processor.run(
    code="source/evaluation.py",
    inputs=[
        ProcessingInput(source=model_data, destination="/opt/ml/processing/model"),
        ProcessingInput(source=preprocessed_test_data, destination="/opt/ml/processing/test"),
    ],
    outputs=[ProcessingOutput(output_name="evaluation", source="/opt/ml/processing/evaluation")],
)
evaluation_job_description = script_processor.jobs[-1].describe()

Now retrieve the file `evaluation.json` from Amazon S3, which contains the evaluation report.

In [ ]:
evaluation_output_config = evaluation_job_description["ProcessingOutputConfig"]
for output in evaluation_output_config["Outputs"]:
    if output["OutputName"] == "evaluation":
        evaluation_s3_uri = output["S3Output"]["S3Uri"] + "/evaluation.json"
        break

evaluation_output = S3Downloader.read_file(evaluation_s3_uri)
evaluation_output_dict = json.loads(evaluation_output)
print(json.dumps(evaluation_output_dict, sort_keys=True, indent=4))